# ZDNABERT

In [1]:
!pip install transformers
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 58.0 MB/s eta 0:00:00


In [2]:
import torch
from torch import nn
import transformers
from transformers import BertTokenizer, BertForTokenClassification
import numpy as np
from Bio import SeqIO
from io import StringIO, BytesIO
# from google.colab import drive, files
from tqdm import tqdm
import pickle
import scipy
from scipy import ndimage

2025-06-17 22:47:36.750661: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750200456.983267      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750200457.051223      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def seq2kmer(seq, k):
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    return kmer

def split_seq(seq, length = 512, pad = 16):
    res = []
    for st in range(0, len(seq), length - pad):
        end = min(st+512, len(seq))
        res.append(seq[st:end])
    return res

def stitch_np_seq(np_seqs, pad = 16):
    res = np.array([])
    for seq in np_seqs:
        res = res[:-pad]
        res = np.concatenate([res,seq])
    return res

In [4]:
model = 'HG kouzine' #@param ["HG chipseq", "HG kouzine", "MM chipseq", "MM kouzine"]
model_confidence_threshold = 0.5 #@param {type:"number"}
minimum_sequence_length = 10 #@param {type:"integer"}

In [5]:
if model == 'HG chipseq':
    model_id = '1VAsp8I904y_J0PUhAQqpSlCn1IqfG0FB'
elif model == 'HG kouzine':
    model_id = '1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx'
elif model == 'MM curax':
    model_id = '1W6GEgHNoitlB-xXJbLJ_jDW4BF35W1Sd'
elif model == 'MM kouzine':
    model_id = '1dXpQFmheClKXIEoqcZ7kgCwx6hzVCv3H'

In [6]:
!gdown $model_id
!gdown 10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
!gdown 16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
!gdown 1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
!gdown 1gZZdtAoDnDiLQqjQfGyuwt268Pe5sXW0


!mkdir 6-new-12w-0
!mv pytorch_model.bin 6-new-12w-0/
!mv config.json 6-new-12w-0/
!mv special_tokens_map.json 6-new-12w-0/
!mv tokenizer_config.json 6-new-12w-0/
!mv vocab.txt 6-new-12w-0/

Downloading...
From (original): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx
From (redirected): https://drive.google.com/uc?id=1dAeAt5Gu2cadwDhbc7OnenUgDLHlUvkx&confirm=t&uuid=1e23f2ee-9031-4021-9e76-97fbf3702cb1
To: /kaggle/working/pytorch_model.bin
100%|█████████████████████████████████████████| 354M/354M [00:01<00:00, 308MB/s]
Downloading...
From: https://drive.google.com/uc?id=10sF8Ywktd96HqAL0CwvlZZUUGj05CGk5
To: /kaggle/working/config.json
100%|██████████████████████████████████████████| 634/634 [00:00<00:00, 3.55MB/s]
Downloading...
From: https://drive.google.com/uc?id=16bT7HDv71aRwyh3gBUbKwign1mtyLD2d
To: /kaggle/working/special_tokens_map.json
100%|███████████████████████████████████████████| 112/112 [00:00<00:00, 491kB/s]
Downloading...
From: https://drive.google.com/uc?id=1EE9goZ2JRSD8UTx501q71lGCk-CK3kqG
To: /kaggle/working/tokenizer_config.json
100%|█████████████████████████████████████████| 40.0/40.0 [00:00<00:00, 207kB/s]
Downloading...
From: https://

In [7]:
tokenizer = BertTokenizer.from_pretrained('6-new-12w-0/')
model = BertForTokenClassification.from_pretrained('6-new-12w-0/')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(4101, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [8]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/022/581/195/GCF_022581195.2_ilHelZeax1.1/GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz
!gunzip ./GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz
!mv GCF_022581195.2_ilHelZeax1.1_genomic.fna genomic.fna

--2025-06-17 22:48:05--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/022/581/195/GCF_022581195.2_ilHelZeax1.1/GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.31, 130.14.250.10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120848234 (115M) [application/x-gzip]
Saving to: ‘GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz’

GCF_022581195.2_ilH 100%[===================>] 115.25M  3.29MB/s    in 44s     

2025-06-17 22:48:49 (2.63 MB/s) - ‘GCF_022581195.2_ilHelZeax1.1_genomic.fna.gz’ saved [120848234/120848234]



In [9]:
with open('genomic.fna', 'rb') as f:
    uploaded={'genomic': f.read()}

In [10]:
out = []
for key in uploaded.keys():
    print(key)
    out.append(key)
    result_dict = {}
    for seq_record in SeqIO.parse(StringIO(BytesIO(uploaded[key]).read().decode('UTF-8')), 'fasta'):
        kmer_seq = seq2kmer(str(seq_record.seq).upper(), 6)
        seq_pieces = split_seq(kmer_seq)
        print(seq_record.name)
        out.append(seq_record.name)
        with torch.no_grad():
            preds = []
            for seq_piece in tqdm(seq_pieces):
                input_ids = torch.LongTensor(tokenizer.encode(' '.join(seq_piece), add_special_tokens=False))
                outputs = torch.softmax(model(input_ids.cuda().unsqueeze(0))[-1],axis = -1)[0,:,1]
                preds.append(outputs.cpu().numpy())
        result_dict[seq_record.name] = stitch_np_seq(preds)



        labeled, max_label = scipy.ndimage.label(result_dict[seq_record.name]>model_confidence_threshold)
        # print('  start     end')
        out.append('  start     end')
        for label in range(1, max_label+1):
            candidate = np.where(labeled == label)[0]
            candidate_length = candidate.shape[0]
            if candidate_length>minimum_sequence_length:
                # print('{:8}'.format(candidate[0]), '{:8}'.format(candidate[-1]))
                out.append('{:8}'.format(candidate[0]) + '{:8}'.format(candidate[-1]))

    with open(key + '.preds.pkl',"wb") as fh:
      pickle.dump(result_dict, fh)
    print()

with open('text_predictions.txt',"w") as fh:
    for item in out:
        fh.write("%s\n" % item)


genomic
NC_061452.1


100%|██████████| 31275/31275 [12:56<00:00, 40.26it/s]


NC_061453.1


100%|██████████| 30367/30367 [12:33<00:00, 40.30it/s]


NC_061454.1


...

100%|██████████| 43/43 [00:01<00:00, 39.95it/s]


NW_025899721.1


100%|██████████| 41/41 [00:01<00:00, 39.96it/s]


NC_061507.1


100%|██████████| 31/31 [00:00<00:00, 39.87it/s]


In [3]:
import pandas as pd
from tqdm import tqdm

results = pd.DataFrame(columns=['Scaffold', 'Start', 'End'])

with open('text_predictions.txt', 'r') as f:
    scaffold = ''
    lines = f.readlines()
    for line in tqdm(lines):
        if line == 'genomic\n':
            continue
        if line.startswith('NW') or line.startswith('NC'):
            scaffold = line[:-1]
            continue
        if line.split()[0] == 'start':
            continue
        if len(line.split()) < 2:
            start = int(line.split()[0][0:8])
            end = int(line.split()[0][8:16])
        else:
            start, end = int(line.split()[0]), int(line.split()[1])
        results.loc[len(results)] = [scaffold, start, end]

results

100%|██████████| 95741/95741 [01:44<00:00, 920.29it/s] 


,Scaffold,Start,End
0,NC_061452.1,3522,3533
1,NC_061452.1,9959,9973
2,NC_061452.1,10574,10589
3,NC_061452.1,13191,13203
4,NC_061452.1,14194,14210
...,...,...,...
95649,NW_025899721.1,7881,7896
95650,NW_025899721.1,8802,8836
95651,NW_025899721.1,8942,8962
95652,NW_025899721.1,12896,12906


In [4]:
results.to_csv('zdnabert_predictions.bed', index=False, header=False, sep='\t')